In [1]:
import pandas as pd
import numpy as np
import os
import glob

In [2]:
#READING IN MAP FILE FOR XIST JUST TO TAKE ITS STRUCTURE, BUT WILL NOT ACTUALLY USE THE DATA

colnames=['nucleotide_number', 'SHAPE_reactivity', 'SE_reactivity', 'nucleotide_identity'] 
df = pd.read_csv("../../../Data/Map files Xist/XIST_exvivo_1M7_rep1.map", sep="\t",names=colnames, header=None,index_col=0)
df.head()

,SHAPE_reactivity,SE_reactivity,nucleotide_identity
nucleotide_number,,,
1,-999.0,0.0,C
2,-999.0,0.0,G
3,-999.0,0.0,G
4,-999.0,0.0,C
5,-999.0,0.0,T


In [3]:
#REMOVE UNNECESSARY COLUMNS
df = df.drop(df[["SHAPE_reactivity","SE_reactivity"]],axis=1)

In [4]:
#Create empty columnS
df["DeltaSHAPE_ex_vivo"] = 0
df["DeltaSHAPE_in_cell"] = 0
#df["DeltaSHAPE"] = 0


In [5]:
df.head()

,nucleotide_identity,DeltaSHAPE_ex_vivo,DeltaSHAPE_in_cell
nucleotide_number,,,
1,C,0,0
2,G,0,0
3,G,0,0
4,C,0,0
5,T,0,0


In [6]:
#READING IN DELTASHAPE INFORMATION, FROM COMPARISONS OF REPLICATES EX VIVO 1 VS IN CELL 1 & EX VIVO 2 VS IN CELL 2  

deltashape = pd.read_csv('../../../Analysis/deltaSHAPE/XIST_exvivo_vs_invivo_reps1both_1M7.txt', sep="\t", header=0, index_col=0)
#invivo = pd.read_csv('../../Data/Map files Xist/XIST_incell_1M7.txt', sep="\t", header=0, index_col=0)


deltashape2 = pd.read_csv('../../../Analysis/deltaSHAPE/XIST_exvivo_vs_invivo_reps2both_IM7.txt', sep="\t", header=0, index_col=0)
#invivo = pd.read_csv('../../Data/Map files Xist/XIST_incell_1M7.txt', sep="\t", header=0, index_col=0)

In [7]:
deltashape2.head()

,Seq,DeltaSHAPE,Z-factor,Std_Score,Smoothed_Data1,Smoothed_Data2,Unsmoothed_Diff,Data1,Data2
Nuc,,,,,,,,,
179,A,-1.525417,0.526480,-1.928230,0.317628,1.843045,-0.808703,0.109179,0.917883
180,A,-1.403042,0.543571,-1.787718,0.199170,1.602212,-3.580023,0.437573,4.017596
181,G,-1.415818,0.525799,-1.802388,0.425973,1.841791,0.179601,0.050757,-0.128844
182,A,-1.232581,0.305541,-1.591995,0.295265,1.527846,-0.847031,0.789590,1.636621
183,T,-1.556549,0.442654,-1.963976,0.292537,1.849086,-3.030313,0.045448,3.075760


In [8]:
#ADD DETALSHAPE INFORMATION TO A NEW DATAFRAME OF RIGHT SIZE i.e. 17918 ROWS x 3 columns AND CALCULATE AVERAGE DELTASHAPE

df_ = pd.DataFrame(index=np.arange(17918), columns=['nucleotide', 'DeltaSHAPE_1','DeltaSHAPE_2'])
df_ = df_.fillna(0) # with 0s rather than NaNs
df_["DeltaSHAPE_1"].loc[deltashape.index] = deltashape["DeltaSHAPE"]
df_["DeltaSHAPE_2"].loc[deltashape2.index] = deltashape2["DeltaSHAPE"]

#ADD AN EXTRA COLUMN FOR CALCULATING AVERAGE OF DELTASHAPE
df_['average_deltaSHAPE'] = 0

#THE IMPORTANT RULE IS: IF ONLY ONE OF THE TWO FILES HAS DELTASHAPE VALUE FOR A GIVEN POSITION, THEN WE 
#DON'T TAKE THE AVERAGE BETWEEN 0 AND THAT VALUE. WE JUST REPORT THE SINGLE VALUE FROM THE ONLY FILE THAT 
#HAS DELTASHAPE INFORMATION FOR THAT NUCLEOTIDE.
for index,row in df_.iterrows():
    if row["DeltaSHAPE_1"] != 0 and row["DeltaSHAPE_2"] == 0:
         df_.loc[row.name,'average_deltaSHAPE'] = row["DeltaSHAPE_1"]
    elif row["DeltaSHAPE_1"] == 0 and row["DeltaSHAPE_2"] != 0:
        df_.loc[row.name,'average_deltaSHAPE'] = row["DeltaSHAPE_2"]
    elif row["DeltaSHAPE_1"] != 0 and row["DeltaSHAPE_2"] != 0:
        a = np.array([row['DeltaSHAPE_1'], row['DeltaSHAPE_2']])
        df_.loc[row.name,'average_deltaSHAPE'] = np.mean(a)



C:\Users\maran\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [9]:
#REMOVING UNNECESSARY COLUMNS FROM THE DATAFRAME CONTAINING THE AVERAGE DELTASHAPE
#MERGE THIS DATAFRAME WITH THE PREVIOUS DATAFRAME 

df.reset_index()

new_df = df.join(df_)

new_df = new_df.fillna(0)
new_df = new_df.drop(new_df[["nucleotide","DeltaSHAPE_1","DeltaSHAPE_2"]],axis=1)

In [10]:
new_df.head()

,nucleotide_identity,DeltaSHAPE_ex_vivo,DeltaSHAPE_in_cell,average_deltaSHAPE
nucleotide_number,,,,
1,C,0,0,0.0
2,G,0,0,0.0
3,G,0,0,0.0
4,C,0,0,0.0
5,T,0,0,0.0


In [11]:
# POPULATE COLUMNS DeltaSHAPE_ex_vivo AND DeltaSHAPE_in_cell BASED ON THE VALUE OF THE AVERAGE DELTASHAPE

for index,row in new_df.iterrows():
    if row["average_deltaSHAPE"] > 0:
        new_df.loc[row.name,'DeltaSHAPE_ex_vivo'] = 1
        new_df.loc[row.name,'DeltaSHAPE_in_cell'] = 0
        #print('positive')
    elif row["average_deltaSHAPE"] < 0:
        new_df.loc[row.name,'DeltaSHAPE_ex_vivo'] = 0
        new_df.loc[row.name,'DeltaSHAPE_in_cell'] = 1
        #print('negative')
    else:
        new_df.loc[row.name,'DeltaSHAPE_ex_vivo'] = 0
        new_df.loc[row.name,'DeltaSHAPE_in_cell'] = 0
        #print('neither')

In [12]:
new_df.head()

,nucleotide_identity,DeltaSHAPE_ex_vivo,DeltaSHAPE_in_cell,average_deltaSHAPE
nucleotide_number,,,,
1,C,0,0,0.0
2,G,0,0,0.0
3,G,0,0,0.0
4,C,0,0,0.0
5,T,0,0,0.0


In [13]:
#GET LIST OF XIST BED FILES

file_list = glob.glob("../../../Data/Bed files Xist/CLIPdb_transcript_coords/*.bed")
for i in file_list:
    print(i)

../../../Data/Bed files Xist/CLIPdb_transcript_coords\CELF1_E-MTAB-414-ERR039244-ERR039245-ERR039247-ERR039249_xist_coords.bed
../../../Data/Bed files Xist/CLIPdb_transcript_coords\CELF1_E-MTAB-414-ERR039246-ERR039248_xist_coords.bed
../../../Data/Bed files Xist/CLIPdb_transcript_coords\FUS_GSE40653-GSM998872_xist_coords.bed
../../../Data/Bed files Xist/CLIPdb_transcript_coords\FUS_GSE40653-GSM998873_xist_coords.bed
../../../Data/Bed files Xist/CLIPdb_transcript_coords\FUS_GSE40653-GSM998874_xist_coords.bed
../../../Data/Bed files Xist/CLIPdb_transcript_coords\HuR_DO-RIP-Seq_xist_coords.bed
../../../Data/Bed files Xist/CLIPdb_transcript_coords\PTBP1_E-MTAB-414-ERR039255-ERR039256_xist_coords.bed
../../../Data/Bed files Xist/CLIPdb_transcript_coords\RBFOX2_SRX355540_xist_coords.bed
../../../Data/Bed files Xist/CLIPdb_transcript_coords\TARDBP_GSE27394-GSM672062_xist_coords.bed
../../../Data/Bed files Xist/CLIPdb_transcript_coords\TARDBP_GSE27394-GSM672063_xist_coords.bed
../../../Data/Be

In [14]:
#ASSIGN A 1 TO EVERY NUCLEOTIDE THAT IS COVERED BY THE BED FILES FOR EACH PROTEIN

proteins = set()
for i in file_list:
    file = os.path.basename(i)
    protein = file.split("_")[0]
    #print(protein)
    proteins.add(protein)
    if protein not in new_df.columns:
        new_df[protein] = 0
    #else:
    data = pd.read_csv(i,sep="\t",header=None,index_col=None,names=["gene",\
                                                                        "start",\
                                                                        "end",\
                                                                        "name",\
                                                                        "coverage",\
                                                                        ".",\
                                                                        "p_value"])
    for j in data.index:
        start = int(data.loc[j,"start"])          
        end = int(data.loc[j,"end"])
        coordinates = np.arange(start,end+1)
        new_df.loc[coordinates,protein] = 1

In [15]:
big_dataframe = new_df.copy()
big_dataframe.head()

,nucleotide_identity,DeltaSHAPE_ex_vivo,DeltaSHAPE_in_cell,average_deltaSHAPE,CELF1,FUS,HuR,PTBP1,RBFOX2,TARDBP
nucleotide_number,,,,,,,,,,
1,C,0,0,0.0,0,0,0,0,0,0
2,G,0,0,0.0,0,0,0,0,0,0
3,G,0,0,0.0,0,0,0,0,0,0
4,C,0,0,0.0,0,0,0,0,0,0
5,T,0,0,0.0,0,0,0,0,0,0


In [16]:
#READING IN DIFF BUM HMM DATA 

df_1 = pd.read_csv("../../../Xist in vivo vs. ex vivo scaled by dc_diff_BUM_HMM_analysis.txt",sep="\t")
#Select columns
df_1 = df_1[['UM','MU']]

#Rename columns
df_1.columns = ['diffBUMHMM_ex_vivo', 'diffBUMHMM_in_cell']

In [17]:
df_1.head()

,diffBUMHMM_ex_vivo,diffBUMHMM_in_cell
1,1.790644e-20,1.693237e-14
2,6.823904e-03,6.789051e-03
3,9.110841e-03,9.031615e-03
4,6.927069e-03,6.793530e-03
5,5.186918e-06,1.128597e-08


In [18]:
#Adding diffBUMHMM data to big dataframe 
big_dataframe ["diffBUMHMM_in_cell"] = 0
big_dataframe ['diffBUMHMM_ex_vivo'] = 0

big_dataframe ["diffBUMHMM_in_cell"].loc[df_1.index] = df_1["diffBUMHMM_in_cell"]
big_dataframe ['diffBUMHMM_ex_vivo'].loc[df_1.index] = df_1['diffBUMHMM_ex_vivo']

big_dataframe.head()

,nucleotide_identity,DeltaSHAPE_ex_vivo,DeltaSHAPE_in_cell,average_deltaSHAPE,CELF1,FUS,HuR,PTBP1,RBFOX2,TARDBP,diffBUMHMM_in_cell,diffBUMHMM_ex_vivo
nucleotide_number,,,,,,,,,,,,
1,C,0,0,0.0,0,0,0,0,0,0,1.693237e-14,1.790644e-20
2,G,0,0,0.0,0,0,0,0,0,0,6.789051e-03,6.823904e-03
3,G,0,0,0.0,0,0,0,0,0,0,9.031615e-03,9.110841e-03
4,C,0,0,0.0,0,0,0,0,0,0,6.793530e-03,6.927069e-03
5,T,0,0,0.0,0,0,0,0,0,0,1.128597e-08,5.186918e-06


In [19]:
#ANNOTATE DRRS PREDICTED BY DSTRUCT, WITH SEARCH LENGTH OF 11 NUCLEOTIDEs, IN THE BIG DTAFARAME  

df_2 = pd.read_csv("../../../Analysis/dStruct/Xist/output_dStruct_Xist_res_table_11nt.txt", sep="\t", header=0)

big_dataframe ["dStruct_DDR"] = 0

#starts= df_2['Start'].values.tolist()
#stops=df_2['Stop'].values.tolist()

for j in df_2.index:
        start = int(df_2.loc[j,"Start"])          
        end = int(df_2.loc[j,"Stop"])
        coordinates = np.arange(start,end+1)
        big_dataframe.loc[coordinates,"dStruct_DDR"] = 1

In [20]:
big_dataframe= big_dataframe.reset_index()
big_dataframe.head()

,nucleotide_number,nucleotide_identity,DeltaSHAPE_ex_vivo,DeltaSHAPE_in_cell,average_deltaSHAPE,CELF1,FUS,HuR,PTBP1,RBFOX2,TARDBP,diffBUMHMM_in_cell,diffBUMHMM_ex_vivo,dStruct_DDR
0,1,C,0,0,0.0,0,0,0,0,0,0,1.693237e-14,1.790644e-20,0
1,2,G,0,0,0.0,0,0,0,0,0,0,6.789051e-03,6.823904e-03,0
2,3,G,0,0,0.0,0,0,0,0,0,0,9.031615e-03,9.110841e-03,0
3,4,C,0,0,0.0,0,0,0,0,0,0,6.793530e-03,6.927069e-03,0
4,5,T,0,0,0.0,0,0,0,0,0,0,1.128597e-08,5.186918e-06,0


In [21]:
df_rawcounts_exvivorep1 = pd.read_csv("../../../Data/XIST_1M7_ex-vivo_rep1.txt", sep="\t", header=0)
df_rawcounts_incellrep1 = pd.read_csv("../../../Data/XIST_1M7_in-cell_rep1.txt", sep="\t", header=0)

df_rawcounts_exvivorep2 = pd.read_csv("../../../Data/XIST_1M7_ex-vivo_rep2.txt", sep="\t", header=0)
df_rawcounts_incellrep2 = pd.read_csv("../../../Data/XIST_1M7_in-cell_rep2.txt", sep="\t", header=0)

df_rawcounts_exvivorep1= df_rawcounts_exvivorep1.drop(df_rawcounts_exvivorep1[["Nucleotide","Sequence"]],axis=1)
df_rawcounts_incellrep1= df_rawcounts_incellrep1.drop(df_rawcounts_incellrep1[["Nucleotide","Sequence"]],axis=1)

df_rawcounts_exvivorep2= df_rawcounts_exvivorep2.drop(df_rawcounts_exvivorep2[["Nucleotide","Sequence"]],axis=1)
df_rawcounts_incellrep2= df_rawcounts_incellrep2.drop(df_rawcounts_incellrep2[["Nucleotide","Sequence"]],axis=1)

df_rawcounts_exvivorep1.columns = ['DMSO_read_exvivo1', 'DMSO_rate_exvivo1', '1M7_read_exvivo1', '1M7_rate_exvivo1']
df_rawcounts_incellrep1.columns = ['DMSO_read_incell1', 'DMSO_rate_incell1', '1M7_read_incell1', '1M7_rate_incell1']

df_rawcounts_exvivorep2.columns = ['DMSO_read_exvivo2', 'DMSO_rate_exvivo2', '1M7_read_exvivo2', '1M7_rate_exvivo2']
df_rawcounts_incellrep2.columns = ['DMSO_read_incell2', 'DMSO_rate_incell2', '1M7_read_incell2', '1M7_rate_incell2']
        
new_dataframe= big_dataframe.join(df_rawcounts_exvivorep1)
new_dataframe = new_dataframe.join(df_rawcounts_incellrep1)

new_dataframe= new_dataframe.join(df_rawcounts_exvivorep2)
new_dataframe = new_dataframe.join(df_rawcounts_incellrep2)

new_dataframe 


counter_rep1 = 0
counter_rep2 =0
counter_rep1and2 = 0
    
for index, row in new_dataframe.iterrows():
    if row['diffBUMHMM_ex_vivo'] > 0.9:
        if row['DMSO_rate_exvivo1'] == 0 and row['1M7_rate_exvivo1'] == 0:  
            counter_rep1=counter_rep1 +1
            print('0 DMSO and 1M7 ex vivo in rep 1: ' +str(row['nucleotide_number']))
            if row['DMSO_rate_exvivo2'] == 0 and row['1M7_rate_exvivo2'] == 0:
                print('0 DMSO and 1M7 ex vivo in both reps 1 and 2: ' +str(row['nucleotide_number']))
                counter_rep1and2=counter_rep1and2 +1
        elif row['DMSO_rate_exvivo2'] == 0 and row['1M7_rate_exvivo2'] == 0:
            print('0 DMSO and 1M7 ex vivo in rep 2: ' +str(row['nucleotide_number']))
            counter_rep2=counter_rep2 + 1
            if row['DMSO_rate_exvivo1'] == 0 and row['1M7_rate_exvivo1'] == 0:
                print('0 DMSO and 1M7 ex vivo in both reps 1 and 2: ' +str(row['nucleotide_number']))
                counter_rep1and2=counter_rep1and2 +1

#print(counter_rep1)
#print(counter_rep2)
#print(counter_rep1and2)  

0 DMSO and 1M7 ex vivo in rep 1: 63
0 DMSO and 1M7 ex vivo in both reps 1 and 2: 63
0 DMSO and 1M7 ex vivo in rep 1: 66
0 DMSO and 1M7 ex vivo in both reps 1 and 2: 66
0 DMSO and 1M7 ex vivo in rep 2: 70
0 DMSO and 1M7 ex vivo in rep 1: 72
0 DMSO and 1M7 ex vivo in rep 2: 77
0 DMSO and 1M7 ex vivo in rep 2: 4587
0 DMSO and 1M7 ex vivo in rep 2: 4589
0 DMSO and 1M7 ex vivo in rep 1: 4590
0 DMSO and 1M7 ex vivo in both reps 1 and 2: 4590
0 DMSO and 1M7 ex vivo in rep 2: 4591
0 DMSO and 1M7 ex vivo in rep 2: 4599
0 DMSO and 1M7 ex vivo in rep 2: 4605
0 DMSO and 1M7 ex vivo in rep 2: 4610
0 DMSO and 1M7 ex vivo in rep 2: 4611
0 DMSO and 1M7 ex vivo in rep 2: 4612
0 DMSO and 1M7 ex vivo in rep 2: 4613
0 DMSO and 1M7 ex vivo in rep 2: 4615
0 DMSO and 1M7 ex vivo in rep 2: 4623
0 DMSO and 1M7 ex vivo in rep 2: 4631
0 DMSO and 1M7 ex vivo in rep 1: 4643
0 DMSO and 1M7 ex vivo in both reps 1 and 2: 4643
0 DMSO and 1M7 ex vivo in rep 1: 7075
0 DMSO and 1M7 ex vivo in rep 1: 9655


In [22]:
#SAVE TO FILE
big_dataframe.to_csv("../../../big_dataframe_Xist_diffBUM_HMM_scaled_dc.txt",sep="\t")